In [22]:
#importing required libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib


In [23]:
# loading  data from the csv file to pandas dataframe
series_data = pd.read_csv('All_Streaming_Shows.csv')
movies_data = pd.read_csv('movies.csv')



In [24]:
series_data

,Series Title,Year Released,Content Rating,IMDB Rating,R Rating,Genre,Description,No of Seasons,Streaming Platform
0,Breaking Bad,2008,18+,9.5,100,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",5Seasons,Netflix
1,Game of Thrones,2011,18+,9.3,99,"Action & Adventure,Drama",Seven noble families fight for control of the ...,8Seasons,"HBO MAX,HBO"
2,Rick and Morty,2013,18+,9.2,97,"Animation,Comedy",Rick is a mentally-unbalanced but scientifical...,4Seasons,"Free Services,HBO MAX,Hulu"
3,Stranger Things,2016,16+,8.8,96,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",3Seasons,Netflix
4,The Boys,2019,18+,8.7,95,"Action & Adventure,Comedy",A group of vigilantes known informally as “The...,2Seasons,Prime Video
...,...,...,...,...,...,...,...,...,...
12348,A Fishing Story with Ronnie Green,2017,NaN,NaN,-1,"2017,Prime Video",A Fishing Story with Ronnie Green has one or m...,2Seasons,"Prime Video,fuboTV"
12349,CMT Most Shocking,2003,NaN,NaN,-1,-1,-1,-1,-1
12350,NHL Road to the Outdoor Classics,2016,NaN,NaN,-1,"2016,Prime Video",Road to the NHL Outdoor Classics takes us deep...,1Season,"Prime Video,Epix"
12351,Addy Media,2018,NaN,NaN,-1,"2018,Prime Video",Addy Media has one or more episodes streaming ...,1Season,Prime Video


In [59]:
# Movie title names in the dataset
movie_title_list = movies_data['title'].tolist()
movie_title_list[0:30] # Taking 30 title names for reference

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall']

In [26]:
cols_to_check = ["Genre", "Series Title", "Description"]
rows_with_minus_one = series_data[series_data[cols_to_check].apply(lambda row: any(str(cell) == '-1' for cell in row), axis=1)]
print(rows_with_minus_one)


                                            Series Title  Year Released  \
6084                                  Lost in Transition           2018   
7855                                  Mariposa de Barrio           2017   
8200                                  Fareed Zakaria GPS           2008   
8243                         Talking with Chris Hardwick           2017   
8321                      Mind Control with Derren Brown           2007   
...                                                  ...            ...   
12340  Who Killed Angie Dodge? Keith Morrison Investi...           2017   
12345                              Get Ready For The UFC           2005   
12346                Mario and Courtney's Wedding Fiesta           2012   
12347                                  SuperCars Exposed           2008   
12349                                  CMT Most Shocking           2003   

      Content Rating  IMDB Rating  R Rating Genre Description No of Seasons  \
6084             NaN

In [27]:
series_data= series_data.drop(rows_with_minus_one.index)
series_data

,Series Title,Year Released,Content Rating,IMDB Rating,R Rating,Genre,Description,No of Seasons,Streaming Platform
0,Breaking Bad,2008,18+,9.5,100,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",5Seasons,Netflix
1,Game of Thrones,2011,18+,9.3,99,"Action & Adventure,Drama",Seven noble families fight for control of the ...,8Seasons,"HBO MAX,HBO"
2,Rick and Morty,2013,18+,9.2,97,"Animation,Comedy",Rick is a mentally-unbalanced but scientifical...,4Seasons,"Free Services,HBO MAX,Hulu"
3,Stranger Things,2016,16+,8.8,96,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",3Seasons,Netflix
4,The Boys,2019,18+,8.7,95,"Action & Adventure,Comedy",A group of vigilantes known informally as “The...,2Seasons,Prime Video
...,...,...,...,...,...,...,...,...,...
12344,"Stop, Breathe & Think Kids: Mindful Games",2017,NaN,NaN,-1,"2017,Hulu",Mindfulness made easy and fun for kids. Discov...,1Season,Hulu
12348,A Fishing Story with Ronnie Green,2017,NaN,NaN,-1,"2017,Prime Video",A Fishing Story with Ronnie Green has one or m...,2Seasons,"Prime Video,fuboTV"
12350,NHL Road to the Outdoor Classics,2016,NaN,NaN,-1,"2016,Prime Video",Road to the NHL Outdoor Classics takes us deep...,1Season,"Prime Video,Epix"
12351,Addy Media,2018,NaN,NaN,-1,"2018,Prime Video",Addy Media has one or more episodes streaming ...,1Season,Prime Video


In [29]:
#Taking input of favorite movie from user
movie_name = input(' Enter your favourite movie name to recommend u the  best  web series: ')

 Enter your favourite movie name to recommend u the  best  web series: spider


In [30]:
#Finding close match to the movie name entered by user with the movie dataset  using  [ difflib ]
close_match = difflib.get_close_matches(movie_name, movie_title_list)
print(close_match)

['Spider', 'Casper', 'Spider-Man']


In [31]:
#Taking the best matched name from user input
input_movie = close_match[0]
print(input_movie)

Spider


In [32]:
# Taking features in dataset which we are using for comparison
Screened_series_data = series_data[['Genre', 'Description', 'Series Title']].copy()
filtered_movies_data = movies_data[movies_data['title'] == input_movie][['genres', 'keywords', 'tagline']].copy()

In [33]:
# Renaming the columns of filtered_movies_data with the column names of series data 
new_column_names = Screened_series_data.columns.tolist()
filtered_movies_data.rename(columns=dict(zip(filtered_movies_data.columns, new_column_names)), inplace=True)

In [34]:
Screened_series_data

,Genre,Description,Series Title
0,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",Breaking Bad
1,"Action & Adventure,Drama",Seven noble families fight for control of the ...,Game of Thrones
2,"Animation,Comedy",Rick is a mentally-unbalanced but scientifical...,Rick and Morty
3,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",Stranger Things
4,"Action & Adventure,Comedy",A group of vigilantes known informally as “The...,The Boys
...,...,...,...
12344,"2017,Hulu",Mindfulness made easy and fun for kids. Discov...,"Stop, Breathe & Think Kids: Mindful Games"
12348,"2017,Prime Video",A Fishing Story with Ronnie Green has one or m...,A Fishing Story with Ronnie Green
12350,"2016,Prime Video",Road to the NHL Outdoor Classics takes us deep...,NHL Road to the Outdoor Classics
12351,"2018,Prime Video",Addy Media has one or more episodes streaming ...,Addy Media


In [35]:
filtered_movies_data


,Genre,Description,Series Title
3085,Drama Mystery Thriller,secret asylum bed and breakfast place past psy...,The only thing worse than losing your mind... ...


In [36]:
#Concatinating both series_data and movies_data
new_data = pd.concat([Screened_series_data, filtered_movies_data], ignore_index=True)

In [37]:
#Renaming column name
new_data = new_data.rename(columns={'Series Title': 'Series_title'})
new_data

,Genre,Description,Series_title
0,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",Breaking Bad
1,"Action & Adventure,Drama",Seven noble families fight for control of the ...,Game of Thrones
2,"Animation,Comedy",Rick is a mentally-unbalanced but scientifical...,Rick and Morty
3,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",Stranger Things
4,"Action & Adventure,Comedy",A group of vigilantes known informally as “The...,The Boys
...,...,...,...
11870,"2017,Prime Video",A Fishing Story with Ronnie Green has one or m...,A Fishing Story with Ronnie Green
11871,"2016,Prime Video",Road to the NHL Outdoor Classics takes us deep...,NHL Road to the Outdoor Classics
11872,"2018,Prime Video",Addy Media has one or more episodes streaming ...,Addy Media
11873,"2014,Free Services",My Dream Derelict Home has one or more episode...,My Dream Derelict Home


In [38]:
#Taking features to compare 
top_features = ['Genre', 'Description', 'Series_title']


In [31]:
#filling missing (NaN) values with an empty string ''.
for feature in top_features:
	new_data[feature] = new_data[feature].fillna('')

In [43]:
# Threading the top features together
feature_thread = new_data['Genre']+' '+new_data['Description']+' '+new_data['Series_title']

In [44]:
#To convert collection of text documents into numerical vectors.
vec_rise = TfidfVectorizer()

In [45]:
# Text data stored in "feature_thread" transformed into TF-IDF vectors. 
vec_features = vec_rise.fit_transform(feature_thread)

In [46]:
#Computes the cosine similarity between each pair of rows (documents) in "vec_features".
sim = cosine_similarity(vec_features)

In [47]:
new_data

,Genre,Description,Series_title
0,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",Breaking Bad
1,"Action & Adventure,Drama",Seven noble families fight for control of the ...,Game of Thrones
2,"Animation,Comedy",Rick is a mentally-unbalanced but scientifical...,Rick and Morty
3,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",Stranger Things
4,"Action & Adventure,Comedy",A group of vigilantes known informally as “The...,The Boys
...,...,...,...
11870,"2017,Prime Video",A Fishing Story with Ronnie Green has one or m...,A Fishing Story with Ronnie Green
11871,"2016,Prime Video",Road to the NHL Outdoor Classics takes us deep...,NHL Road to the Outdoor Classics
11872,"2018,Prime Video",Addy Media has one or more episodes streaming ...,Addy Media
11873,"2014,Free Services",My Dream Derelict Home has one or more episode...,My Dream Derelict Home


In [48]:
index_of_the_movie= new_data.index[-1] #as we added movie to the last index in the dataframe
index_of_the_movie

11874

In [58]:
#Create a list of tuples representing the similarity values between a specific movie (input from user) and all other series in a similarity list.
sim_score = list(enumerate(sim[index_of_the_movie]))


In [56]:
# Sorting sim_score in incresing order
Top_similar_series= sorted(sim_score, key = lambda x:x[1], reverse = True) 
Top_similar_series[:20]

[(11874, 1.0),
 (5259, 0.2659197275502342),
 (3740, 0.25504574749896863),
 (10985, 0.1939766699511546),
 (10927, 0.18261184629360172),
 (8681, 0.17946555049300547),
 (10809, 0.1769728406377836),
 (11734, 0.1706421537228378),
 (6597, 0.1684799838623207),
 (5971, 0.1648925560745572),
 (7053, 0.16229527750365708),
 (3927, 0.14940652821581785),
 (8760, 0.1443262770014157),
 (1919, 0.1420543955615805),
 (3464, 0.14002855300183667),
 (1692, 0.13546295038078252),
 (5488, 0.13299832130328149),
 (2958, 0.13102494380338198),
 (3941, 0.12423589306909918),
 (10814, 0.12264648559375435)]

In [53]:
Screened_series_data.shape

(11874, 3)

In [54]:
#Printing the Top 10 series 
print("Top 10 recommended series ")
print()
j=1
for i in Top_similar_series:
    if j==11:
        break
    if i[0]==11874:
        continue
    print(j," ",series_data.loc[i[0], "Series Title"])
    j+=1

Top 10 recommended series 

1   Hyori's Bed and Breakfast
2   Kakuriyo -Bed & Breakfast for Spirits-
3   Tasty 101
4   Secrets of Your Mind
5   Immortal Egypt
6   Crimes that Made History
7   Zumba Awesome Legs & Booty System
8   Shattered
9   Re:Mind
10   Hot Ones: The Game Show
